In [20]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import tensorboard as tb
import datetime, os

In [21]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


In [22]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
# Importando Data a utilizar en el proyecto:

data_total = np.load('C:/Users/Gary/Google Drive/Universidad/MAESTRIA/Data Science/Año 1/1. Trimestre I/2. Ciencia de Datos en Python/Tareas/Proyecto Final/proyecto_data/proyecto_training_data.npy')
data_total.shape

#Se usará el 80% para los "Datos de entrenamiento"
Parte1 = round(len(data_total)*0.8)
data_entrenamiento = data_total[0:int(data_total.shape[0]*.8)+1]

#Se usará el 20% para "Datos de validación y pruebas"
Parte2 = round(len(data_total)*0.2)
data_validacion = data_total[int(data_total.shape[0]*.8)+1:]



In [5]:
def error(y_real,y_aprox):
  return 1/2*tf.reduce_mean(tf.math.square(y_real - y_aprox) )

In [6]:
x = data_entrenamiento[:,3]
y = data_entrenamiento[:,0]
tamaño_muestra = data_entrenamiento.shape[0]


In [66]:
class ModeloLineal:
  def __init__(self):
    tf.reset_default_graph()
    self.mb = tf.get_variable("PENDIENTE_INTERC",dtype=tf.float64,shape=[2,1],initializer=tf.zeros_initializer())
    self.error = tf.get_variable("ERROR_VAL",dtype=tf.float64,shape=[1],initializer=tf.zeros_initializer())
    
  def __call__(self,x):
    x = tf.concat([x,tf.ones_like(x,name="XVec_1")], axis=1,name="XAdd_1")  
    return tf.matmul(x,self.mb,name="Prediccion")

  def actualizar(self,x,y,learning_rate):
    prediccion = self(x)
    error = 1/2 * tf.math.reduce_mean(tf.math.square(y - prediccion,name="Error_Error"),axis=0,name="Error_Med") 
    gradiente = tf.gradients(error,self.mb, name="Grad")  
    actual_error = tf.assign(self.error, error)
    actual_param = tf.assign(self.mb,self.mb - tf.multiply(gradiente,learning_rate)[0],name="Actualizacion_Param")

    return actual_param,actual_error

In [67]:
#Modificando nuestra data de entrenamiento para acoplarla
#y definiendo los hiper parametros
Data_X = data_entrenamiento[:,1].reshape([-1,1])
Data_Y = data_entrenamiento[:,0].reshape([-1,1])

#Hyper-Parametros
lr = 0.00001
bs = 350
ep = 350

print(Data_Y)

[[208500.]
 [181500.]
 [223500.]
 ...
 [245350.]
 [173000.]
 [235000.]]


In [68]:
#Funcion modelo regresion lineal

def ModeloRegresionLineal(X,Y,learning_rate,batch_size,epochs):  
    #plt.scatter(X,Y)

    modelo = ModeloLineal()
    tamaño_muestra = X.shape[0]

    total_iteraciones =  int(tamaño_muestra/batch_size)

    #Creacion de tensores a utilizar
    tensor_x = tf.placeholder(tf.float64,[None,1],"tensor_x")
    tensor_y = tf.placeholder(tf.float64,[None,1],"tensor_y")

    #Creacion Modelo
    prediccion = modelo(tensor_x)
    actualizacion_parametros = modelo.actualizar(tensor_x,tensor_y,learning_rate)

    #Inicio de programancion de grafo
    with tf.train.MonitoredSession() as session:
        tf.reset_default_graph()   
        log_dir = "logs/Entrenamiento/"+"BS-"+str(batch_size)+"_"+"LR-"+str(learning_rate)+"_"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        Grafo = tf.summary.FileWriter(log_dir, session.graph)


        for epoch in range(epochs):    
            for i in range(total_iteraciones):          
                muestra_inicio = i*batch_size
                muestra_fin = muestra_inicio + batch_size

                x_mb =  np.array(X[muestra_inicio:muestra_fin])
                y_mb = np.array(Y[muestra_inicio:muestra_fin])
                feed_dict = {tensor_x:x_mb, tensor_y:y_mb}
                _,mb,e = session.run([actualizacion_parametros,modelo.mb,modelo.error],feed_dict=feed_dict)            
                summary = tf.Summary(value=[tf.Summary.Value(tag="Error", simple_value=e[0])])
                Grafo.add_summary(summary, epoch)

            feed_dict = {tensor_x:X, tensor_y:Y}
        PendienteInter_final = session.run([modelo.mb],feed_dict=feed_dict)
        
        Grafo.close()



In [70]:
ModeloRegresionLineal(Data_X,Data_Y,0.0001,50,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [71]:
ModeloRegresionLineal(Data_X,Data_Y,0.0001,100,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [72]:
ModeloRegresionLineal(Data_X,Data_Y,0.0001,200,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [73]:
ModeloRegresionLineal(Data_X,Data_Y,0.00001,50,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [74]:
ModeloRegresionLineal(Data_X,Data_Y,0.00001,100,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [75]:
ModeloRegresionLineal(Data_X,Data_Y,0.00001,200,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [76]:
ModeloRegresionLineal(Data_X,Data_Y,0.000001,50,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [78]:
ModeloRegresionLineal(Data_X,Data_Y,0.000001,100,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [79]:
ModeloRegresionLineal(Data_X,Data_Y,0.000001,200,300)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [80]:
%tensorboard --logdir logs/Entrenamiento

Reusing TensorBoard on port 6006 (pid 1260), started 4:21:22 ago. (Use '!kill 1260' to kill it.)

## Conclusión: ##



### Mientras mas bajo sea el learning rate y menor es nuestro tamaño del batch size el error mas alto nos queda el error.
### Por lo que lo reconendable es utilizar un modelo con un lr bajo con altas iteracionas para que el entrenamiento del modelo no nos genere mayor error y que tenga un comportamiento predictivo que podamos utilizar. 
